In [3]:
import os
import numpy as np
import tensorflow as tf
import tensorflow_datasets as tfds
import matplotlib.pyplot as plt
from sklearn.metrics import confusion_matrix
import seaborn as sns

ImportError: cannot import name '_imaging' from 'PIL' (/usr/lib/python3/dist-packages/PIL/__init__.py)

In [5]:
train_ds, test_ds , val_ds= tfds.load('eurosat/rgb', split=['train[:60%]', 'train[60%:80%]', 'train[80%:]'], shuffle_files=False, as_supervised=True)

2022-12-17 09:57:24.762923: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [6]:
BUFFER_SIZE = 1000
BATCH_SIZE = 32
img_height=64
img_width =64
IMG_SIZE = (img_height, img_width)
def resize_image(images, labels):
    images = tf.image.resize(images, IMG_SIZE)
    return images, labels

In [7]:
train_ds = train_ds.map(resize_image).cache().shuffle(BUFFER_SIZE)\
    .batch(BATCH_SIZE).prefetch(buffer_size=tf.data.AUTOTUNE)

val_ds = val_ds.map(resize_image).batch(BATCH_SIZE)\
    .prefetch(buffer_size=tf.data.AUTOTUNE)

test_ds = test_ds.map(resize_image).batch(BATCH_SIZE)\
    .prefetch(buffer_size=tf.data.AUTOTUNE)

In [8]:
from tensorflow.keras import layers
from tensorflow.keras import models

tf.keras.utils.set_random_seed(42)

model = models.Sequential([
    layers.Rescaling(1./255, input_shape=(img_height, img_width, 3)),
    layers.Conv2D(32, 3, activation='relu'),
    layers.MaxPooling2D(),
    layers.Conv2D(64, 3, activation='relu'),
    layers.MaxPooling2D(),
    layers.Conv2D(128, 3, activation='relu'),
    layers.MaxPooling2D(),
    layers.Flatten(),
    layers.Dense(128, activation='relu'),
    layers.Dense(10)
])

In [9]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 rescaling (Rescaling)       (None, 64, 64, 3)         0         
                                                                 
 conv2d (Conv2D)             (None, 62, 62, 32)        896       
                                                                 
 max_pooling2d (MaxPooling2D  (None, 31, 31, 32)       0         
 )                                                               
                                                                 
 conv2d_1 (Conv2D)           (None, 29, 29, 64)        18496     
                                                                 
 max_pooling2d_1 (MaxPooling  (None, 14, 14, 64)       0         
 2D)                                                             
                                                                 
 conv2d_2 (Conv2D)           (None, 12, 12, 128)       7

In [10]:
# Compile the model
model.compile(optimizer='adam',
                loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
                metrics=['accuracy'])

In [11]:
history = model.fit(train_ds, epochs=15, validation_data=val_ds)

Epoch 1/15
507/507 [==============================] - 32s 61ms/step - loss: 1.2946 - accuracy: 0.4953 - val_loss: 0.9429 - val_accuracy: 0.6719
Epoch 2/15
507/507 [==============================] - 32s 63ms/step - loss: 0.8642 - accuracy: 0.6831 - val_loss: 0.7462 - val_accuracy: 0.7331
Epoch 3/15
507/507 [==============================] - 35s 70ms/step - loss: 0.6966 - accuracy: 0.7468 - val_loss: 0.5619 - val_accuracy: 0.8002
Epoch 4/15
507/507 [==============================] - 33s 65ms/step - loss: 0.6241 - accuracy: 0.7730 - val_loss: 0.6116 - val_accuracy: 0.7706
Epoch 5/15
507/507 [==============================] - 35s 69ms/step - loss: 0.5170 - accuracy: 0.8115 - val_loss: 0.4804 - val_accuracy: 0.8289
Epoch 6/15
507/507 [==============================] - 33s 66ms/step - loss: 0.4709 - accuracy: 0.8301 - val_loss: 0.5373 - val_accuracy: 0.8161
Epoch 7/15
507/507 [==============================] - 35s 68ms/step - loss: 0.4134 - accuracy: 0.8552 - val_loss: 0.4878 - val_accuracy:

In [12]:
test_loss, test_acc = model.evaluate(test_ds)
print('Test accuracy: ', test_acc)

169/169 [==============================] - 2s 14ms/step - loss: 0.6138 - accuracy: 0.8341
Test accuracy:  0.834074079990387


In [13]:
acc = history.history['accuracy']
val_acc = history.history['val_accuracy']
loss = history.history['loss']
val_loss = history.history['val_loss']

epochs = range(1, len(acc) + 1)

plt.plot(epochs, acc, 'bo', label='Training acc')
plt.plot(epochs, val_acc, 'b', label='Validation acc')
plt.title('Training and validation accuracy')
plt.legend()

plt.figure()

plt.plot(epochs, loss, 'bo', label='Training loss')
plt.plot(epochs, val_loss, 'b', label='Validation loss')
plt.title('Training and validation loss')
plt.legend()

plt.show()

NameError: name 'plt' is not defined

In [ ]:
from sklearn.metrics import confusion_matrix
import seaborn as sns

y_pred = model.predict(test_ds)
y_pred = np.argmax(y_pred, axis=1)
y_true = []
for _, label in test_ds:
    y_true.append(label.numpy())
y_true = np.concatenate(y_true)

cm = confusion_matrix(y_true, y_pred)
cm = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]

plt.figure(figsize=(10, 10))
sns.heatmap(cm, annot=True, fmt='.2f', xticklabels=labels, yticklabels=labels)
plt.ylabel('True label')
plt.xlabel('Predicted label')
plt.show()